<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
### Due 22.10
#### Identifying the suspects credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: NaN
* Num_Credit_Inquiries: NaN
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fun by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [1]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [3]:
df.info()
suspects.info()


<class 'pandas.core.frame.DataFrame'>
Index: 29223 entries, 0 to 49998
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       29223 non-null  int64  
 1   Occupation                29223 non-null  object 
 2   Annual_Income             29223 non-null  float64
 3   Monthly_Inh_Salary        29223 non-null  float64
 4   Num_Bank_Accounts         29223 non-null  int64  
 5   Num_Credit_Card           29223 non-null  int64  
 6   Interest_Rate             29223 non-null  int64  
 7   Num_of_Loan               29223 non-null  int64  
 8   Delay_from_due_date       29223 non-null  int64  
 9   Num_of_Delayed_Payment    29223 non-null  int64  
 10  Changed_Credit_Limit      29223 non-null  float64
 11  Num_Credit_Inquiries      29223 non-null  float64
 12  Credit_Mix                29223 non-null  object 
 13  Outsting_Debt             29223 non-null  float64
 14  Credit_Util

In [4]:
df["Credit_Mix"].value_counts()

Credit_Mix
Standard    13421
Good         8963
Bad          6839
Name: count, dtype: int64

# 1. Preparing the data
## 1.1 Data cleaning
 Perform OHE over the "Occupation" feature

 Then, perform LE over Payment_of_Min_Amount and Payment_Behaviour

 _hint: As we will be testing only one model no need to define a pipeline_

In [5]:

# Your code here:
ohe = OneHotEncoder(sparse= False)
df_fitted = ohe.fit_transform(df[['Occupation']])
categories = ohe.get_feature_names_out(input_features=['Occupation'])
df_encoded = pd.DataFrame(df_fitted, columns=categories, index= df.index)
# Replace the 'Occupation' column with the one-hot encoded columns
df_encoded = pd.concat([df.drop(['Occupation'], axis=1), df_encoded], axis=1)

le = LabelEncoder()
df_encoded['Payment_of_Min_Amount'] = le.fit_transform(df_encoded['Payment_of_Min_Amount'])
df_encoded['Payment_Behaviour'] = le.fit_transform(df_encoded['Payment_Behaviour'])

print(df_encoded)






       Age  Annual_Income  Monthly_Inh_Salary  Num_Bank_Accounts  \
0       23       19114.12         1824.843333                  3   
1       24       19114.12         1824.843333                  3   
3       24       19114.12         4182.004291                  3   
5       28       34847.84         3037.986667                  2   
8       35      143162.64         4182.004291                  1   
...    ...            ...                 ...                ...   
49990   50       37188.10         3097.008333                  1   
49992   29       20002.88         1929.906667                 10   
49993   29       20002.88         1929.906667                 10   
49997   25       39628.99         3359.415833                  4   
49998   25       39628.99         4182.004291                  4   

       Num_Credit_Card  Interest_Rate  Num_of_Loan  Delay_from_due_date  \
0                    4              3            4                    3   
1                    4           

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 1.2 Dataset splitting

Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

Then perform :
* OneHotEncoding over the **CreditMix** feature.
* A MinMaxScaller over the independent features

In [6]:
# Your code here:
X = pd.DataFrame(df_encoded.drop(columns=['Credit_Mix']))
y = pd.DataFrame(df_encoded['Credit_Mix'])
#print(y.index)
y_fitted = ohe.fit_transform(y[['Credit_Mix']])
categories_credit_mix = ohe.get_feature_names_out(input_features=['Credit_Mix'])
y = pd.DataFrame(y_fitted, columns=categories_credit_mix, index= y.index)
# Replace the 'Occupation' column with the one-hot encoded columns
#y = pd.concat([y.drop(['Credit_mix'], axis=1), y], axis=1)
print(y)

#Define the scaler
scaler = MinMaxScaler()
#Fit the scaler
X = scaler.fit_transform(X)
#X = pd.DataFrame(X, columns=)
print(X)

       Credit_Mix_Bad  Credit_Mix_Good  Credit_Mix_Standard
0                 0.0              1.0                  0.0
1                 0.0              1.0                  0.0
3                 0.0              1.0                  0.0
5                 0.0              1.0                  0.0
8                 0.0              1.0                  0.0
...               ...              ...                  ...
49990             0.0              1.0                  0.0
49992             1.0              0.0                  0.0
49993             1.0              0.0                  0.0
49997             0.0              1.0                  0.0
49998             0.0              1.0                  0.0

[29223 rows x 3 columns]
[[5.71772166e-02 5.01784709e-04 1.02087051e-01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.72865420e-02 5.01784709e-04 1.02087051e-01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.72865420e-02 5.01784709e-04 2.60275285e-01 ... 1.000

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### 1.2.1 Train Test splitting
Now split the data in X_train, X_test, y_train, y_test, 

You can use test_size = 0.2 and a random_state of 42

In [7]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23378, 42) (5845, 42) (23378, 3) (5845, 3)


### 1.2.2 final touches
Convert your datasets to `Torch tensors` of type `torch.float`

In [8]:
#Your code here:
X_train = torch.tensor(X_train.values, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
X_test = torch.tensor(X_test.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)
print(X_train.size(), y_train.size(), y_test.size(), X_test.size())

torch.Size([23378, 42]) torch.Size([23378, 3]) torch.Size([5845, 3]) torch.Size([5845, 42])


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
You can set the number of neurons to 150.

In [9]:
# Define a neural network class here:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

# Define the input and output sizes
D_in =  X_train.shape[1]
D_out =  y_train.shape[1]


In [10]:
# Instantiate your model here
model1 = Net(D_in, 150, D_out)
print(model1)
pytorch_total_params = sum(p.numel() for p in model1.parameters() if p.requires_grad)
print(pytorch_total_params)

Net(
  (linear1): Linear(in_features=42, out_features=150, bias=True)
  (linear2): Linear(in_features=150, out_features=3, bias=True)
  (activation): ReLU()
)
6903


## 2.2 finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://anvilproject.org/guides/content/creating-links), [BCEWithLogitsLoss](hhttps://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
* `iterations` : 150, 250, 500
* `learning rate` : 0.00005, 0.001, 12.031


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [11]:
# Define your loss function here:
criterion = nn.CrossEntropyLoss()
# Define your SGD optimizer for finding the weights of the network here
optimizer = torch.optim.SGD(model1.parameters(), lr=0.000005)

In [12]:
# Perform your iterations here
losses1 = []
losses1_test = []

for t in range(150):                # 500 iterations

    # Forward pass: compute prediction on training set
    y_pred = model1(X_train)

    # Compute loss
    loss = criterion(y_pred, y_train)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break

    # Compute gradient
    optimizer.zero_grad()
    loss.backward()

    # Update
    optimizer.step()

    # Compute loss on test set
    losses1_test.append(criterion(model1(X_test), y_test).item())

0 1.1057748794555664
1 1.1057744026184082
2 1.1057738065719604
3 1.1057732105255127
4 1.1057727336883545
5 1.1057721376419067
6 1.105771541595459
7 1.1057710647583008
8 1.105770468711853
9 1.1057699918746948
10 1.105769395828247
11 1.1057686805725098
12 1.1057682037353516
13 1.1057674884796143
14 1.105767011642456
15 1.1057665348052979
16 1.1057660579681396
17 1.1057653427124023
18 1.1057648658752441
19 1.1057642698287964
20 1.1057636737823486
21 1.1057631969451904
22 1.1057626008987427
23 1.105762004852295
24 1.1057615280151367
25 1.1057608127593994
26 1.1057603359222412
27 1.1057597398757935
28 1.1057592630386353
29 1.1057586669921875
30 1.1057580709457397
31 1.105757474899292
32 1.1057569980621338
33 1.1057565212249756
34 1.1057558059692383
35 1.1057552099227905
36 1.1057547330856323
37 1.1057541370391846
38 1.1057536602020264
39 1.105752944946289
40 1.1057524681091309
41 1.1057519912719727
42 1.1057512760162354
43 1.1057507991790771
44 1.1057502031326294
45 1.1057497262954712
46 1.

## 2.3 Model Accuracy
Identify the models accuracy over the train and test parts of the training dataset

In [13]:
# deactivate dropout layers

# Training accuracy


# Test accuracy


# 3. Predictions over the suspects dataset
## 3.1 Retrain a new model over the full training dataset
#### Please use the following parameters for this section:
* ``neurons`` = 150
* ``learning`` rate = 0.00005
* ``criterion`` = CrossEntropyLoss
* `iterations` = 500

_hint you may have to redo some preprocessing as you did in part one_

In [14]:
#preprocessing: X = training and suspects is testing dataset
suspects = suspects.drop(columns=['userID'])
df_full_train = pd.DataFrame(df_encoded.drop(columns=['Credit_Mix']))
df_full_test = pd.DataFrame(X)
suspects_train = pd.DataFrame(suspects)

df_full_train = torch.tensor(df_full_train.values, dtype=torch.float)
df_full_test = torch.tensor(df_full_test.values, dtype=torch.float)
suspects_train = torch.tensor(suspects.values, dtype=torch.float)

print(df_full_train.shape, df_full_test.shape, suspects_train.shape, suspects_train)

torch.Size([29223, 42]) torch.Size([29223, 42]) torch.Size([714, 42]) tensor([[2.3000e+01, 1.9114e+04, 1.8248e+03,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.4000e+01, 1.9114e+04, 1.8248e+03,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.4000e+01, 1.9114e+04, 4.1820e+03,  ..., 0.0000e+00, 1.0000e+00,
         0.0000e+00],
        ...,
        [3.4000e+01, 1.5736e+04, 1.0433e+03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.2000e+01, 2.0503e+04, 4.1820e+03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.2000e+01, 2.0503e+04, 1.5886e+03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


In [15]:
# Define a new model here:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        
        # Define the layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input layer to hidden layer
        self.relu = nn.ReLU()  # ReLU activation function
        self.fc2 = nn.Linear(hidden_size, output_size)  # Hidden layer to output layer
        

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

# Define the input and output sizes
input_size =  df_full_train.shape[1]
output_size =  df_full_test.shape[1]
hidden_size = 150  # 150 neurons in the hidden layer

model2 = Net(input_size, hidden_size, output_size)
print(model2)


Net(
  (fc1): Linear(in_features=42, out_features=150, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=150, out_features=42, bias=True)
)


In [16]:
# Define your MSE loss here:
criterion = nn.CrossEntropyLoss()

# Define your SGD optimizer for finding the weights of the network here:
optimizer = torch.optim.SGD(model2.parameters(), lr=0.00005)


In [17]:
# perform your training here
losses1 = []
losses1_test = []

for t in range(500):                # 500 iterations

    # Forward pass: compute prediction on training set
    y_pred = model2(df_full_train)

    # Compute loss
    loss = criterion(y_pred, df_full_test)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break

    # Compute gradient
    optimizer.zero_grad()
    loss.backward()

    # Update
    optimizer.step()

    # Compute loss on test set
    #losses1_test.append(criterion(model2(X_test), y_test).item())

AttributeError: 'Net' object has no attribute 'activation'

## 3.2 Predict over the suspects dataset

In [ ]:
# Predict which users have a good credit score here:
